In [3]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(42)
np.random.seed(42)

In [5]:
def normalize(alpha):
    return (alpha-1)/9
def denormalize(alpha_t):
    return alpha_t**9+1

n_collocation = 10000
x_colloc=torch.rand(n_collocation,1)
alpha_colloc=1+9*torch.rand(n_collocation,1)
alpha_colloc_norm=normalize(alpha_colloc)

n_boundary=1000
alpha_bc=1+9*torch.rand(n_boundary,1)  #scaling alpha to [1,10]
alpha_bc_norm=normalize(alpha_bc)

x_bc_left=torch.zeros_like(alpha_bc)    #x=0
x_bc_right=torch.ones_like(alpha_bc)    #x=1

In [6]:
class ParametricPINN(nn.Module):
    def __init__(self, layers=(2, 64, 64, 64, 64, 1)):
        super().__init__()
        net_layers = []
        for in_dim, out_dim in zip(layers[:-2], layers[1:-1]):
            net_layers.append(nn.Linear(in_dim, out_dim))
            net_layers.append(nn.Tanh())
            net_layers.append(nn.Linear(layers[-2], layers[-1]))
            self.network = nn.Sequential(*net_layers)
    def forward(self, x, alpha_norm):
        inputs = torch.cat([x, alpha_norm], dim=1)
        return self.network(inputs)
pinn = ParametricPINN()

In [2]:
def pde_residual(model, x, alpha, alpha_norm):
    """Compute u_xx + alpha."""
    x.requires_grad_(True)
    alpha_norm.requires_grad_(False)
    u = model(x, alpha_norm)
    # First derivative wrt x
    u_x = autograd.grad(u, x, grad_outputs=torch.ones_like(u),create_graph=True)[0]
    # Second derivative wrt x
    u_xx = autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x),create_graph=True)[0]
    return u_xx + alpha # residual should be ~0